In [1]:
import shutil
import os
import numpy as np
import argparse

# Data split


## path + calculates how many train data per class

### !set path

In [2]:
def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

# Path to data
path_to_data = "./data/video/train_oulu"
# Path to test data where to save
path_to_test_data = "./data/video/test_oulu"
# Train ratio - 0.7 means splitting data in 70 % train and 30 % test
train_ratio = float(0.7)

# get dirs
_, dirs, _ = next(os.walk(path_to_data))

# calculates how many train data per class
data_counter_per_class = np.zeros((len(dirs)))
for i in range(len(dirs)):
    path = os.path.join(path_to_data, dirs[i])
    files = get_files_from_folder(path)
    data_counter_per_class[i] = len(files)
print("data per class", data_counter_per_class)
test_counter = np.round(data_counter_per_class * (1 - train_ratio))
print("data per test", test_counter)

data per class [240. 240. 240. 240. 240. 240.]
data per test [72. 72. 72. 72. 72. 72.]


## transfers files

In [3]:
# transfers files
for i in range(len(dirs)):
    path_to_original = os.path.join(path_to_data, dirs[i])
    path_to_save = os.path.join(path_to_test_data, dirs[i])

    #creates dir
    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)
    files = get_files_from_folder(path_to_original)
                                 
    # random file sequence
    np.random.shuffle(files)
                                  
    # moves data
    for j in range(int(test_counter[i])):
        dst = os.path.join(path_to_save, files[j])
        src = os.path.join(path_to_original, files[j])
        shutil.move(src, dst)



# Face alignment part
## Preprocess video to frame

In [4]:
import subprocess
import os
import threading
import pdb
import copy

In [5]:
def run_threads(threads, n_thread):
    used_thread = []
    for num, new_thread in enumerate(threads):
        print('thread index: {:}'.format(num), end=' \t')
        new_thread.start()
        used_thread.append(new_thread)
        
        if num % n_thread == 0:
            for old_thread in used_thread:
                old_thread.join()
            used_thread = []

class threadFun(threading.Thread):
    def __init__(self, func, args):
        super(threadFun, self).__init__()
        self.fun = func
        self.args = args
    def run(self):
        self.fun(*self.args)

In [6]:
VIDEO_EXTENSIONS = ['mp4', 'webm', 'avi']

# check file name, is it end with video type extension
def is_video_file(filename):
    return any(filename.endswith(extension) for extension in VIDEO_EXTENSIONS)

# helper function to make a directory if not exist
def makefile(file_dir):
    if not os.path.exists(file_dir):
        os.makedirs(file_dir)

# use ffmpeg to extract video to frame to the frame_output
def video2frame(video_input, frame_output):
    linux_commod = 'ffmpeg -i {:} -f image2 {:}/%07d.jpg'.format(video_input, frame_output)
    print('{:}'.format(video_input))
    subprocess.getstatusoutput(linux_commod)

In [7]:
def run_vdo2frame(video_dir, frame_dir, n_thread):
    print('Starting: convert videos into frames\nvideo_dir: {:}\tframe_dir: {:}'.format(video_dir, frame_dir))
    threads = []
    for root, dirs, files in os.walk(video_dir):
        for file_name in files:
            if is_video_file(file_name):
                # get video name and path
                video_name = os.path.join(root, file_name)
                # create frame output path from changing video directory in video_name to frame_dir and split the file extension
                frame_output_path = os.path.splitext(video_name.replace(video_dir, frame_dir))[0]
                makefile(frame_output_path)
                threads.append(threadFun(video2frame, (video_name, frame_output_path)))
    run_threads(threads, n_thread)
    print('all threads is finished') 

### !set path
Do on both train and test data

In [8]:
video_dir_train ='./data/video/train_oulu'
frame_dir_train = './data/frame/train_oulu'
run_vdo2frame(video_dir_train, frame_dir_train, n_thread =20)

Starting: convert videos into frames
video_dir: ./data/video/train_oulu	frame_dir: ./data/frame/train_oulu
thread index: 0 	./data/video/train_oulu\Angry\NI_P001_D_A.avi
thread index: 1 	./data/video/train_oulu\Angry\NI_P001_S_A.avi
thread index: 2 	./data/video/train_oulu\Angry\NI_P001_W_A.avi
thread index: 3 	./data/video/train_oulu\Angry\NI_P002_S_A.avi
thread index: 4 	./data/video/train_oulu\Angry\NI_P002_W_A.avi
thread index: 5 	./data/video/train_oulu\Angry\NI_P003_D_A.avi
thread index: 6 	./data/video/train_oulu\Angry\NI_P003_S_A.avi
thread index: 7 	./data/video/train_oulu\Angry\NI_P003_W_A.avi
thread index: 8 	./data/video/train_oulu\Angry\NI_P004_D_A.avi
thread index: 9 	./data/video/train_oulu\Angry\NI_P004_S_A.avi
thread index: 10 	./data/video/train_oulu\Angry\NI_P004_W_A.avi
thread index: 11 	./data/video/train_oulu\Angry\NI_P005_D_A.avi
thread index: 12 	./data/video/train_oulu\Angry\NI_P005_W_A.avi
thread index: 13 	./data/video/train_oulu\Angry\NI_P006_D_A.avi
thread 

In [9]:
video_dir_test ='./data/video/test_oulu'
frame_dir_test = './data/frame/test_oulu'
run_vdo2frame(video_dir_test, frame_dir_test, n_thread = 20)

Starting: convert videos into frames
video_dir: ./data/video/test_oulu	frame_dir: ./data/frame/test_oulu
thread index: 0 	./data/video/test_oulu\Angry\NI_P002_D_A.avi
thread index: 1 	./data/video/test_oulu\Angry\NI_P005_S_A.avi
thread index: 2 	./data/video/test_oulu\Angry\NI_P006_S_A.avi
thread index: 3 	./data/video/test_oulu\Angry\NI_P006_W_A.avi
thread index: 4 	./data/video/test_oulu\Angry\NI_P007_W_A.avi
thread index: 5 	./data/video/test_oulu\Angry\NI_P008_D_A.avi
thread index: 6 	./data/video/test_oulu\Angry\NI_P009_S_A.avi
thread index: 7 	./data/video/test_oulu\Angry\NI_P009_W_A.avi
thread index: 8 	./data/video/test_oulu\Angry\NI_P010_D_A.avi
thread index: 9 	./data/video/test_oulu\Angry\NI_P011_D_A.avi
thread index: 10 	./data/video/test_oulu\Angry\NI_P011_W_A.avi
thread index: 11 	./data/video/test_oulu\Angry\NI_P012_S_A.avi
thread index: 12 	./data/video/test_oulu\Angry\NI_P014_W_A.avi
thread index: 13 	./data/video/test_oulu\Angry\NI_P015_W_A.avi
thread index: 14 	./dat

## Face localization

In [10]:
def frame2face(func_path, predictor_path, image_root_folder, save_root_folder, cnn_face_detector, gpu_id=0):

    linux_command = 'python {:} {:} {:} {:} {:} {:}'.format(func_path, predictor_path, 
                                                            image_root_folder, save_root_folder, cnn_face_detector, gpu_id)
    print('{:}'.format(image_root_folder))
    subprocess.getstatusoutput(linux_command)

In [11]:
def run_frame2face(frame_dir, face_dir, n_thread):
    threads = []
    # function
    func_path = './data/face_alignment_code/lib/face_align_cuda.py'
    # Model
    predictor_path      = './data/face_alignment_code/lib/shape_predictor_5_face_landmarks.dat'
    cnn_face_detector   = './data/face_alignment_code/lib/mmod_human_face_detector.dat' 
    for category in os.listdir(frame_dir):
        # create category directory
        category_dir = os.path.join(frame_dir, category)
        for frame_file in os.listdir(category_dir):
            # get frame file name and path for each frame
            frame_root_folder = os.path.join(category_dir, frame_file)
            # create output path for face localization
            face_root_folder = frame_root_folder.replace(frame_dir, face_dir)
            if os.path.isdir(frame_root_folder):
                makefile(face_root_folder)
                threads.append(threadFun(frame2face, (func_path, predictor_path, frame_root_folder, face_root_folder, cnn_face_detector)))

    run_threads(threads, n_thread)
    print('all is over')

### !set path

In [12]:
frame_dir_train = './data/frame/train_oulu'
face_dir_train  = './data/face/train_oulu'
run_frame2face(frame_dir_train, face_dir_train, n_thread=20)

thread index: 0 	./data/frame/train_oulu\Angry\NI_P001_D_A
thread index: 1 	./data/frame/train_oulu\Angry\NI_P001_S_A
thread index: 2 	./data/frame/train_oulu\Angry\NI_P001_W_A
thread index: 3 	./data/frame/train_oulu\Angry\NI_P002_S_A
thread index: 4 	./data/frame/train_oulu\Angry\NI_P002_W_A
thread index: 5 	./data/frame/train_oulu\Angry\NI_P003_D_A
thread index: 6 	./data/frame/train_oulu\Angry\NI_P003_S_A
thread index: 7 	./data/frame/train_oulu\Angry\NI_P003_W_A
thread index: 8 	./data/frame/train_oulu\Angry\NI_P004_D_A
thread index: 9 	./data/frame/train_oulu\Angry\NI_P004_S_A
thread index: 10 	./data/frame/train_oulu\Angry\NI_P004_W_A
thread index: 11 	./data/frame/train_oulu\Angry\NI_P005_D_A
thread index: 12 	./data/frame/train_oulu\Angry\NI_P005_W_A
thread index: 13 	./data/frame/train_oulu\Angry\NI_P006_D_A
thread index: 14 	./data/frame/train_oulu\Angry\NI_P007_D_A
thread index: 15 	./data/frame/train_oulu\Angry\NI_P007_S_A
thread index: 16 	./data/frame/train_oulu\Angry\NI

In [13]:
frame_dir_test = './data/frame/test_oulu' 
face_dir_test  = './data/face/test_oulu'
run_frame2face(frame_dir_test, face_dir_test, n_thread=20)

thread index: 0 	./data/frame/test_oulu\Angry\NI_P002_D_A
thread index: 1 	./data/frame/test_oulu\Angry\NI_P005_S_A
thread index: 2 	./data/frame/test_oulu\Angry\NI_P006_S_A
thread index: 3 	./data/frame/test_oulu\Angry\NI_P006_W_A
thread index: 4 	./data/frame/test_oulu\Angry\NI_P007_W_A
thread index: 5 	./data/frame/test_oulu\Angry\NI_P008_D_A
thread index: 6 	./data/frame/test_oulu\Angry\NI_P009_S_A
thread index: 7 	./data/frame/test_oulu\Angry\NI_P009_W_A
thread index: 8 	./data/frame/test_oulu\Angry\NI_P010_D_A
thread index: 9 	./data/frame/test_oulu\Angry\NI_P011_D_A
thread index: 10 	./data/frame/test_oulu\Angry\NI_P011_W_A
thread index: 11 	./data/frame/test_oulu\Angry\NI_P012_S_A
thread index: 12 	./data/frame/test_oulu\Angry\NI_P014_W_A
thread index: 13 	./data/frame/test_oulu\Angry\NI_P015_W_A
thread index: 14 	./data/frame/test_oulu\Angry\NI_P016_D_A
thread index: 15 	./data/frame/test_oulu\Angry\NI_P016_S_A
thread index: 16 	./data/frame/test_oulu\Angry\NI_P017_W_A
thread 

## Create text file for train list and label

 Example Tree-Structured Data Directory (same as test data) <br> 
── train_data <br>
   ├── Angry <br>
   ├── Disgust <br>
   ├── Fear <br>
   ├── Happy <br>
   ├── Neutral <br>
   ├── Sad <br>
   └── Surprise

In [14]:
def get_txt(dir_path, output_path):
    # Open the output file in write mode
    with open(output_path, "w") as f:
    # Walk through the directory
        for root, dirs, files in os.walk(dir_path):
            # Skip if no files in the directory
            if not files:
                continue
            # Get the parent directory name as the label
            label = os.path.basename(os.path.dirname(root))
            # Write the directory path (relative to dir_path) and label to the output file
            relative_path = os.path.relpath(root, dir_path)
            relative_path = relative_path.replace("\\", "/")
            f.write(f"{relative_path} {label}\n")

### !set path

In [15]:
# Train

# Define the directory to scan
train_dir_path = "./data/face/train_oulu"

# Define the output file path
train_output_file_path = "./data/txt/train_oulu.txt"

get_txt(train_dir_path, train_output_file_path)

### !set path

In [16]:
# Test

# Define the directory to scan
test_dir_path = "./data/face/test_oulu"

# Define the output file path
test_output_file_path = "./data/txt/test_oulu.txt"

get_txt(test_dir_path, test_output_file_path)